#Single-Fidelity GP Surrogate (Multi-Output)

In [1]:
from IPython.display import clear_output

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"
clear_output()

In [4]:
!pip install emukit
clear_output()
print('Emukit installed')

Emukit installed


In [5]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
%matplotlib inline

np.random.seed(20)

In [6]:
# Emukit imports
import GPy
import emukit.multi_fidelity
import emukit.test_functions
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
import emukit.test_functions.multi_fidelity

In [7]:
!ls "/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation"

data_generation_50percent.ipynb  High-Fidelity
data_generation.ipynb		 Low-Fidelity


In [8]:
#High-Fidelity (HF) and Low-Fidelity (LF) data
X_hf = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/High-Fidelity/CM3/input_data_permeability_50percent.npy')
Y_hf = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/High-Fidelity/CM3/output_data_permeability_50percent.npy')

X_lf = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/Low-Fidelity/CM3/input_data_200_permeability_50percent.npy')
Y_lf = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/Low-Fidelity/CM3/output_data_200_permeability_50percent.npy')
#Y_lf = Y_lf[:,:2] #pequeña correción para dejar Y_lf con 2 columnas, esto se debe arreglar

# 1 Additional samples for HF and LF, with the particularity that each Xi is the same for both, could serve as a Testing or Validation Data
X_hf_100 = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/High-Fidelity/CM3/X_hf_100_new.npy')
Y_hf_100 = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/High-Fidelity/CM3/Y_hf_100_new.npy')

X_lf_100 = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/Low-Fidelity/CM3/X_lf_100_new.npy')
Y_lf_100 = np.load('/content/drive/My Drive/Colab Notebooks/UQ Lung/1 Generated Data from Simulations/data_generation/Low-Fidelity/CM3/Y_lf_100_new.npy')

print("X HF size :", X_hf.shape)
print("Y HF size :", Y_hf.shape)
print("X LF size :", X_lf.shape)
print("Y LF size :", Y_lf.shape)
print("")
print("Additional X HF size :", X_hf_100.shape)
print("Additional Y HF size :", Y_hf_100.shape)
print("Additional X LF size :", X_lf_100.shape)
print("Additional Y LF size :", Y_lf_100.shape)

X HF size : (20, 6)
Y HF size : (20, 2)
X LF size : (200, 6)
Y LF size : (200, 2)

Additional X HF size : (100, 6)
Additional Y HF size : (100, 2)
Additional X LF size : (100, 6)
Additional Y LF size : (100, 2)


In [9]:
from sklearn.model_selection import train_test_split
# Next the LF and HF data is split into Training and Testing set
X_lf_train, X_lf_test = train_test_split(X_lf, train_size=0.95, shuffle=True, random_state=1) # this split the input data
Y_lf_train, Y_lf_test = train_test_split(Y_lf, train_size=0.95, shuffle=True, random_state=1) # this split the output data

X_hf_train, X_hf_test = train_test_split(X_hf, train_size=0.95, shuffle=True, random_state=1) # this split the input data
Y_hf_train, Y_hf_test = train_test_split(Y_hf, train_size=0.95, shuffle=True, random_state=1) # this split the output data

X_hf_test_100 = X_hf_100
Y_hf_test_100 = Y_hf_100

X_lf_test_100 = X_lf_100
Y_lf_test_100 = Y_lf_100

In [10]:
from sklearn.preprocessing import StandardScaler

## Normalize input data
scaler = StandardScaler()

X_lf_train = scaler.fit_transform(X_lf_train)
X_lf_test = scaler.transform(X_lf_test)
X_lf_test_100 = scaler.transform(X_lf_test_100)

X_hf_train = scaler.fit_transform(X_hf_train)
X_hf_test = scaler.transform(X_hf_test)
X_hf_test_100 = scaler.transform(X_hf_test_100)

In [11]:
input_dim = X_hf_train.shape[1]
Y_hf_train_r = Y_hf_train[:,0][:,np.newaxis]
Y_hf_train_c = Y_hf_train[:,1][:,np.newaxis]

## Construct a single-fidelity model
kernel_r = GPy.kern.RBF(input_dim, ARD = True)
kernel_c = GPy.kern.RBF(input_dim, ARD = True)
m_r = GPy.models.GPRegression(X_hf_train, Y_hf_train_r, kernel_r)
m_c = GPy.models.GPRegression(X_hf_train, Y_hf_train_c, kernel_c)
#print(m)
#m.optimize(optimizer='scg', max_iters=1000)
m_r.optimize()
m_c.optimize()
#print(m)

 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


In [12]:
Y_pred_mean_R, Y_pred_var_R = m_r.predict(X_hf_test_100)
Y_pred_mean_C, Y_pred_var_C = m_c.predict(X_hf_test_100)

In [13]:
from sklearn.metrics import mean_squared_error

In [14]:
# Export values
np.save('gp_C_50percent.npy', Y_pred_mean_C)
np.save('gp_R_50percent.npy', Y_pred_mean_R)

In [ ]:
#split_list = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

#split_list = [0.5, 0.55, 0.57, 0.6, 0.65, 0.67, 0.7, 0.75, 0.77, 0.8, 0.85, 0.87, 0.9, 0.95, 0.97]
split_list = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
#split_list = [0.5, 0.6]
#random_list = [1,2,3,4,5,6,7,8,9,10]
random_list = np.arange(1, 100+1,10)
RMSE_list_R = []
RMSE_list_C = []
std_R = []
std_C = []

for split in split_list:
  print("Train split: ", split)
  RMSE_inner_list_R = []
  RMSE_inner_list_C = []
  for random_number in random_list:
    # Split
    X_lf_train, X_lf_test = train_test_split(X_lf, train_size=split, shuffle=True, random_state=random_number) # this split the input data
    Y_lf_train, Y_lf_test = train_test_split(Y_lf, train_size=split, shuffle=True, random_state=random_number) # this split the output data

    X_hf_train, X_hf_test = train_test_split(X_hf, train_size=split, shuffle=True, random_state=random_number) # this split the input data
    Y_hf_train, Y_hf_test = train_test_split(Y_hf, train_size=split, shuffle=True, random_state=random_number) # this split the output data

    X_hf_test_100 = X_hf_100
    Y_hf_test_100 = Y_hf_100

    X_lf_test_100 = X_lf_100
    Y_lf_test_100 = Y_lf_100


    # Normalize
    scaler = StandardScaler()

    X_lf_train = scaler.fit_transform(X_lf_train)
    X_lf_test = scaler.transform(X_lf_test)
    X_lf_test_100 = scaler.transform(X_lf_test_100)

    X_hf_train = scaler.fit_transform(X_hf_train)
    X_hf_test = scaler.transform(X_hf_test)
    X_hf_test_100 = scaler.transform(X_hf_test_100)

    print(X_hf_train.shape)

    # Train
    input_dim = X_hf_train.shape[1]
    Y_hf_train_r = Y_hf_train[:,0][:,np.newaxis]
    Y_hf_train_c = Y_hf_train[:,1][:,np.newaxis]

    ## Construct a single-fidelity model
    kernel_r = GPy.kern.RBF(input_dim, ARD = True)
    kernel_c = GPy.kern.RBF(input_dim, ARD = True)
    m_r = GPy.models.GPRegression(X_hf_train, Y_hf_train_r, kernel_r)
    m_c = GPy.models.GPRegression(X_hf_train, Y_hf_train_c, kernel_c)
    #print(m)
    #m.optimize(optimizer='scg', max_iters=1000)
    m_r.optimize()
    m_c.optimize()
    #print(m)


    # Evaluate
    Y_pred_mean_r, Y_pred_var_r = m_r.predict(X_hf_test_100)
    Y_pred_mean_c, Y_pred_var_c = m_c.predict(X_hf_test_100)
    # H-F predictions
    print("H-F predictions")
    mse_R = mean_squared_error(Y_hf_test_100[:,0], Y_pred_mean_r)
    print('\nMSE R: {}'.format(mse_R))
    mse_C = mean_squared_error(Y_hf_test_100[:,1], Y_pred_mean_c)
    print('\nMSE C: {}'.format(mse_C))
    RMSE_inner_list_R.append(np.sqrt(mse_R))
    RMSE_inner_list_C.append(np.sqrt(mse_C))
  RMSE_list_R.append(np.average(RMSE_inner_list_R))
  RMSE_list_C.append(np.average(RMSE_inner_list_C))
  print(RMSE_inner_list_R)
  print(RMSE_inner_list_C)
  std_R.append(np.std(RMSE_inner_list_R))
  std_C.append(np.std(RMSE_inner_list_C))

#clear_output()

for i in range(len(split_list)):
  print("Train split: ", split_list[i], "||    MSE R: ", RMSE_list_R[i], "||    MSE C: ", RMSE_list_C[i], "||    std R: ", std_R[i], "||    std C: ", std_C[i])

Train split:  0.5
(10, 6)


 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide


H-F predictions

MSE R: 0.0010695670215943654

MSE C: 75.31276907482498
(10, 6)
H-F predictions

MSE R: 0.0007160566329660437

MSE C: 258.11325385663923
(10, 6)


 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


H-F predictions

MSE R: 0.0016909315366035937

MSE C: 84.96203387732442
(10, 6)


...

 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


H-F predictions

MSE R: 0.000460522999330124

MSE C: 1.8164984986937502
(19, 6)


 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


H-F predictions

MSE R: 0.00031672462821018114

MSE C: 0.9228254172868656
(19, 6)
H-F predictions

MSE R: 0.00010002216705541868

MSE C: 62.708861370541854
[0.009540100072823082, 0.009540596136119453, 0.016536186539098024, 0.010603914119302171, 0.015357025175305477, 0.013491905173731514, 0.00907371029958475, 0.021459799610670273, 0.017796758924314874, 0.010001108291355448]
[8.343901056634849, 8.343901056630642, 0.9606478152568771, 0.9886472907581253, 1.4476387763647156, 1.5219834222304938, 1.1142386871856547, 1.3477753888143789, 0.9606380261507794, 7.918892685883668]
Train split:  0.5 ||    MSE R:  0.028970362405650613 ||    MSE C:  9.280266608889722 ||    std R:  0.006312290706144892 ||    std C:  3.244811298283962
Train split:  0.55 ||    MSE R:  0.030095204336445552 ||    MSE C:  8.39303176575318 ||    std R:  0.016426694678814618 ||    std C:  5.0873221133352935
Train split:  0.6 ||    MSE R:  0.024359646682543208 ||    MSE C:  6.143345029084872 ||    std R:  0.006664270372426524 |

 /usr/local/lib/python3.8/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide


In [ ]:
plt.plot(np.arange(10,20),RMSE_list_C, marker = 'o')
plt.fill_between(np.arange(10,20),np.asarray(RMSE_list_C) - np.asarray(std_C),np.asarray(RMSE_list_C) + np.asarray(std_C),alpha=.2)

In [ ]:
plt.plot(np.arange(10,20),RMSE_list_R, marker = 'o')
plt.fill_between(np.arange(10,20),np.asarray(RMSE_list_R) - np.asarray(std_R),np.asarray(RMSE_list_R) + np.asarray(std_R),alpha=.2)

In [ ]:
# Export values
np.save('RMSE_R_sfgp.npy', RMSE_list_R)
np.save('RMSE_C_sfgp.npy', RMSE_list_C)
np.save('std_R_sfgp.npy', std_R)
np.save('std_C_sfgp.npy', std_C)